# Bernoulli manifold: Analytic validation for geodesics and NGD

This notebook validates numeric geodesics and natural gradient descent (NGD) on the 1D Bernoulli statistical manifold under the Fisher–Rao metric, comparing against closed-form analytic solutions in Fisher-angle coordinates. It follows the core theory in the repository document 'CFT+ULCC Framework Implementation.pdf'. We use:
- Fisher–Rao metric g(θ) = 1/(θ(1−θ))
- Fisher angle ζ(θ) = 2·arcsin(√θ)

Geodesics are straight lines in ζ, and NGD with quadratic potential V(ζ)=0.5(ζ−ζ*)² integrates exactly to exponential decay in ζ. We compare these analytics to simple numeric integrators in θ with small steps, avoiding domain boundaries by keeping θ ∈ [0.05, 0.95]. Runtime is kept under 5 seconds.


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from pathlib import Path

# Print/display setup
np.set_printoptions(precision=6, suppress=True)

# Artifacts path
artifacts_dir = Path('spec-first/artifacts/figures')
artifacts_dir.mkdir(parents=True, exist_ok=True)

# Deterministic RNG if needed
rng = np.random.default_rng(0)

def clip_theta(theta, lo=0.05, hi=0.95, eps=1e-6):
    """Clip θ to a safe interior; returns numpy array or scalar consistently."""
    return np.clip(theta, max(lo, eps), min(1.0 - eps, hi))


In [ ]:
import numpy as _np  # local alias not used elsewhere; keep numpy as np

def fisher_metric_bernoulli(theta):
    """Fisher–Rao metric g(θ)=1/(θ(1−θ)) for Bernoulli."""
    th = np.asarray(theta, dtype=float)
    return 1.0 / (th * (1.0 - th))

def fisher_angle(theta):
    """Fisher angle ζ(θ) = 2*arcsin(sqrt(θ))."""
    th = clip_theta(np.asarray(theta, dtype=float))
    return 2.0 * np.arcsin(np.sqrt(th))

def inv_fisher_angle(zeta):
    """Inverse Fisher angle: θ(ζ) = sin(ζ/2)^2."""
    z = np.asarray(zeta, dtype=float)
    return np.sin(0.5 * z) ** 2

def distance_fr(theta0, theta1):
    """Closed-form Fisher–Rao distance between θ0 and θ1."""
    th0 = float(theta0); th1 = float(theta1)
    return 2.0 * abs(np.arcsin(np.sqrt(th1)) - np.arcsin(np.sqrt(th0)))

def gprime_closed(theta):
    """Closed-form derivative g'(θ) = (2θ-1)/(θ^2 (1−θ)^2)."""
    th = np.asarray(theta, dtype=float)
    return (2.0 * th - 1.0) / (th**2 * (1.0 - th)**2)

def gprime_numerical(theta, h=1e-6):
    """Numerical derivative of g with clipping for robustness."""
    th = np.asarray(theta, dtype=float)
    thp = clip_theta(th + h)
    thm = clip_theta(th - h)
    return (fisher_metric_bernoulli(thp) - fisher_metric_bernoulli(thm)) / (2.0 * h)

def dzetadtheta(theta):
    """dζ/dθ = 1/sqrt(θ(1−θ))."""
    th = clip_theta(np.asarray(theta, dtype=float))
    return 1.0 / np.sqrt(th * (1.0 - th))

def gamma_1d(theta):
    """Christoffel Γ^θ_{θθ} = 0.5 * g^{-1} * g'(θ) for 1D Bernoulli.
    Using closed form, Γ(θ) = (2θ−1)/(2 θ (1−θ))."""
    th = np.asarray(theta, dtype=float)
    ginv = th * (1.0 - th)
    return 0.5 * ginv * gprime_closed(th)


In [ ]:
# Numeric geodesic integration vs analytic Fisher-angle line
theta0 = 0.2
theta1 = 0.8
T = 1.0
N = 2000
dt = T / N
t = np.linspace(0.0, T, N+1)

theta_num = np.empty(N+1, dtype=float)
v = np.empty(N+1, dtype=float)

theta_num[0] = theta0
zeta0 = fisher_angle(theta0)
zeta1 = fisher_angle(theta1)
vz = (zeta1 - zeta0) / T
v[0] = vz * np.sqrt(theta0 * (1.0 - theta0))

for i in range(N):
    th = theta_num[i]
    vel = v[i]
    # 1D geodesic acceleration: a = -Gamma(th) * vel^2
    a = -gamma_1d(th) * (vel ** 2)
    v[i+1] = vel + dt * a
    theta_next = th + dt * v[i+1]
    theta_num[i+1] = clip_theta(theta_next)

# Analytic geodesic in zeta is linear
zeta_lin = zeta0 + (zeta1 - zeta0) * (t / T)
theta_ana = inv_fisher_angle(zeta_lin)

# Errors
max_abs_err_geod = float(np.max(np.abs(theta_num - theta_ana)))

# Discrete Fisher–Rao length
sqrt_g = 1.0 / np.sqrt(theta_num[:-1] * (1.0 - theta_num[:-1]))
L_numeric = float(np.sum(sqrt_g * np.abs(np.diff(theta_num))))
L_analytic = float(distance_fr(theta0, theta1))
rel_len_err = abs(L_numeric - L_analytic) / L_analytic

# Plot and save
fig, ax = plt.subplots(figsize=(6, 3.5))
ax.plot(t, theta_ana, label='analytic θ(t)', lw=2)
ax.plot(t, theta_num, '--', label='numeric θ(t)', lw=1.5)
ax.set_xlabel('t')
ax.set_ylabel('θ')
ax.set_title('Bernoulli geodesic: numeric vs analytic')
ax.legend(loc='best')
fig.tight_layout()
geod_png = artifacts_dir / 'bernoulli_geodesic.png'
fig.savefig(geod_png, dpi=150)
plt.close(fig)


In [ ]:
# NGD: analytic vs numeric Euler in θ
k = 1.0
theta0_ngd = 0.2
theta_star = 0.6
zeta_star = fisher_angle(theta_star)
T2 = 2.0
N2 = 4000
dt2 = T2 / N2
t2 = np.linspace(0.0, T2, N2+1)

theta_ngd = np.empty(N2+1, dtype=float)
zeta_ngd = np.empty(N2+1, dtype=float)
theta_ngd[0] = theta0_ngd
zeta_ngd[0] = fisher_angle(theta_ngd[0])

# Analytic in zeta
zeta0_ngd = zeta_ngd[0]
zeta_ana = zeta_star + (zeta0_ngd - zeta_star) * np.exp(-k * t2)
theta_ana_ngd = inv_fisher_angle(zeta_ana)

for i in range(N2):
    th = theta_ngd[i]
    zt = fisher_angle(th)
    dtheta_dt = -k * (zt - zeta_star) * np.sqrt(th * (1.0 - th))
    theta_next = th + dt2 * dtheta_dt
    theta_ngd[i+1] = clip_theta(theta_next)
    zeta_ngd[i+1] = fisher_angle(theta_ngd[i+1])

# Final-time errors
theta_rel_err_final = abs(theta_ngd[-1] - theta_ana_ngd[-1]) / max(1e-12, abs(theta_ana_ngd[-1]))
zeta_abs_err_final = abs(zeta_ngd[-1] - zeta_ana[-1])

# Plot and save (zeta domain)
fig2, ax2 = plt.subplots(figsize=(6, 3.5))
ax2.plot(t2, zeta_ana, label='analytic ζ(t)', lw=2)
ax2.plot(t2, zeta_ngd, '--', label='numeric ζ(t)', lw=1.5)
ax2.set_xlabel('t')
ax2.set_ylabel('ζ')
ax2.set_title('Bernoulli NGD in Fisher angle: numeric vs analytic')
ax2.legend(loc='best')
fig2.tight_layout()
ngd_png = artifacts_dir / 'bernoulli_ngd.png'
fig2.savefig(ngd_png, dpi=150)
plt.close(fig2)


## Results summary
The summary below prints the key errors and performs acceptance checks.


In [ ]:
print('Geodesic max_abs_err:', f'{max_abs_err_geod:.3e}')
print('Fisher distance L (analytic):', f'{L_analytic:.6f}')
print('Fisher distance L (numeric): ', f'{L_numeric:.6f}', 'rel_err=', f'{rel_len_err:.3e}')
print('NGD final θ relative error:', f'{theta_rel_err_final:.3e}')
print('NGD final ζ absolute error:', f'{zeta_abs_err_final:.3e}')

assert max_abs_err_geod < 1e-3, f'Geodesic max_abs_err {max_abs_err_geod:.3e} >= 1e-3'
assert rel_len_err < 1e-3, f'FR length relative error {rel_len_err:.3e} >= 1e-3'
assert theta_rel_err_final < 1e-2, f'θ relative error {theta_rel_err_final:.3e} >= 1e-2'
assert zeta_abs_err_final < 1e-3, f'ζ absolute error {zeta_abs_err_final:.3e} >= 1e-3'

print('\nAll acceptance checks passed.')
